In [1]:
from data import load_data, data_prep
from train import train, plot_learning_curve, objective
from data_openml import DataSetCatCon
from torch.utils.data import DataLoader
from models import SAINT
import torch.optim as optim
from sklearn.feature_selection import SelectFromModel
from catboost import CatBoostClassifier
import pandas as pd
import optuna
from optuna.trial import Trial


X1, y = load_data(1000, 500)

# select best fingerprints 
fgpts = X1.iloc[:, 2:-1]
model2 = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0)
model2.fit(fgpts, y)
fgpts_selector = SelectFromModel(model2, prefit=True, threshold="mean", max_features=100)
X = pd.concat([X1.iloc[:, :2], X1.loc[:, fgpts.columns[fgpts_selector.get_support()]]], axis=1)

cat_dims, cat_idxs, con_idxs, X_train, y_train, X_valid, y_valid, X_test, y_test, train_mean, train_std, continuous_mean_std = data_prep(X, y, datasplit=[.65, .15, .2])

train_ds = DataSetCatCon(X_train, y_train, cat_idxs,'reg',continuous_mean_std)
trainloader = DataLoader(train_ds, batch_size=256, shuffle=True,num_workers=4)

valid_ds = DataSetCatCon(X_valid, y_valid, cat_idxs,'reg', continuous_mean_std)
validloader = DataLoader(valid_ds, batch_size=256, shuffle=False,num_workers=4)

test_ds = DataSetCatCon(X_test, y_test, cat_idxs,'reg', continuous_mean_std)
testloader = DataLoader(test_ds, batch_size=256, shuffle=False,num_workers=4)

# select best hyperparameters
study = optuna.create_study()
study.optimize(lambda trial: objective(trial, cat_dims, con_idxs, trainloader, validloader, testloader), n_trials=100)
optim_params = study.best_params 

model = SAINT(
    categories = tuple(cat_dims), 
    num_continuous = len(con_idxs),                
    dim = optim_params['dim'],                         
    dim_out = 1,                       
    depth = optim_params['depth'],                        
    heads = optim_params['heads'],                         
    attn_dropout = optim_params['attn_dropout'],             
    ff_dropout = optim_params['ff_dropout'],                  
    mlp_hidden_mults = optim_params['mlp_hidden_mults'],       
    cont_embeddings = 'MLP',
    attentiontype = optim_params['attentiontype'],
    final_mlp_style = optim_params['final_mlp_style'],
    y_dim = 1 # porque es regression 
)

optimizer = optim.AdamW(model.parameters(),lr=optim_params['lr'])
scheduler = 'cosine' # default 


epochs = 100

valid_rmse, test_rmse = train(model, optimizer, scheduler, epochs, trainloader, validloader, testloader)

plot_learning_curve(valid_rmse, test_rmse)


[I 2024-06-18 12:04:53,267] A new study created in memory with name: no-name-ab60d3da-39a7-4cb4-900d-0075476bc5f1
C:\Users\carlo\anaconda3\Lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (4, 2) which is of type tuple.
  warnings.warn(message)
C:\Users\carlo\anaconda3\Lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (8, 4) which is of type tuple.
  warnings.warn(message)
C:\Users\carlo\anaconda3\Lib\site-packages\optuna\distributions.py:524: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (16, 8) which is of type tuple.
  warnings.warn(message)
[W 2024-06-18 12:05:11,560] Trial 0 failed with parameters: {'dim': 25, 'depth': 

AttributeError: 'SAINT' object has no attribute 'transformer'